In [4]:
import os
import datetime
import json

base_directory = "H:\\Financial_Data\\Historical_data"
roots = ["SPXW", "SPY", "UVXY","SPX","QQQ","VIX","CAT"]  # Example roots, adjust as necessary
#roots = ["CAT"]
start_date = datetime.datetime(2022, 5, 11)
end_date = datetime.datetime.now()
date_list = [(start_date + datetime.timedelta(days=x)).strftime('%Y%m%d') for x in range((end_date - start_date).days + 1)]

def find_matching_contracts_and_log_success(roots, base_directory, date_list):
    all_matching_files = []  # To store all matching filenames across roots and dates
    all_contract_details = []
    for root in roots:
        for date in date_list:
            quotes_dir = os.path.join(base_directory, root, 'QUOTES', date)
            iv_greeks_dir = os.path.join(base_directory, root, 'IV_Greeks', date)
            success_dir = os.path.join(base_directory, root, 'Strike_History_Success', date)

            # Ensure directories exist
            if not os.path.exists(quotes_dir) or not os.path.exists(iv_greeks_dir):
                continue  # Skip to the next date if directories do not exist

            # List all files in directories for the current date
            quotes_files = set(os.listdir(quotes_dir))
            iv_greeks_files = set(os.listdir(iv_greeks_dir))

            # Find matching contract names
            matching_files = quotes_files.intersection(iv_greeks_files)
            all_matching_files.extend([(date, file) for file in matching_files])  # Store with date for reference

            # Process matching filenames and prepare for JSON logging
            for filename in matching_files:
                # Extract information from filename
                parts = filename.split('_')
                if len(parts) == 5:  # Assuming filename format is root_start_date_expiration_strike_right.json
                    contract_info = {
                        "root": parts[0],
                        "expiration": parts[2],
                        "strike": parts[3],
                        "right": parts[4].split('.')[0],  # Remove file extension
                        "start_date": parts[1],
                        "end_date":parts[1], 
                    }
                    # Here you can log to JSON or do further processing
                    #print(contract_info)  # For demonstration
                    all_contract_details.append(contract_info)
    

     # Group by root and start_date, then save
    for contract_info in all_contract_details:
        success_folder_path = os.path.join(base_directory, contract_info["root"], 'Strike_History_Success', contract_info["start_date"])
        if not os.path.exists(success_folder_path):
            os.makedirs(success_folder_path)

        file_path = os.path.join(success_folder_path, 'Completed_log.json')
        
        # Check if the file exists and load existing data if it does
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                #print(file_path)
                data = json.load(file)
        else:
            data = []

        # Add contract info if it's not already in the file
        if contract_info not in data:
            data.append(contract_info)
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
    
    return

# Call the function
matching_files = find_matching_contracts_and_log_success(roots, base_directory, date_list)
#print(matching_files)

In [5]:
print("complete")

complete
